In [3]:
%load_ext autoreload
%autoreload 2
from napari.qt.threading import thread_worker
import control
import napari
import gui
import time
import numpy as np

In [5]:
config = {
    "valves": {
        "ip": "192.168.1.2",
        "device": "wago_valves"
    },
    "mux": {
        "device": "mux",
        "mapping": {
            "0_0": 47,
            "0_1": 46,
            "1_0": 45,
            "1_1": 44,
            "2_0": 43,
            "2_1": 42,
            "3_0": 41,
            "3_1": 40,
            "waste": 37,
            "io": 36,
        }
    },
    "camera": {
        "device": "zyla_camera"
    },
    "objective": {
        "device": "ti_objective",
        "states": ["a", "b", "c", "d", "e", "f"]
    },
    "light": {
        "device": "sola_light",
        "port": "COM4"
    },
    "up_filter": {
        "device": "ti_filter1",
        "states": ["mcherry", "egfp", "open1", "cy5", "brightfield", "open2"]
    },
    "down_filter": {
        "device": "ti_filter2"
    },
    "stage": {
        "device": "asi_stage",
        "port": "COM5"
    },
    "focus": {
        "device": "ti_focus"
    },
    "lightpath": {
        "device": "ti_lightpath"
    },
    "uv_filter": {
        "device": "lambda_filter1",
        "port": "COM3",
        "states": ["uv", "open1", "filter-2", "315nm", "filter-4", "340nm", "filter-6", "open2", "filter-8", "brightfield"],
    },
    "right_filter": {
        "device": "lambda_filter2",
        "port": "COM3",
        "states": ["open", "435", "474", "536", "546", "572", "620", "630", "650", "780"],
    },
    "laser": {
        "device": "lambda_shutter1",
        "port": "COM3"
    },
    "uv_shutter": {
        "device": "lambda_shutter2",
        "port": "COM3"
    },
}
c = control.load(config)
c._core.setTimeoutMs(100000)
c._core.setCameraDevice("camera")
c.exposure = 100

In [7]:
c.close()

NameError: name 'c' is not defined

In [73]:
c.valves[36] = 1

In [22]:
c.exposure = 1000

In [64]:
c.uv_shutter = "closed"

sys:1: RuntimeWarning: RuntimeError in aborted thread: Error in device "camera":  Snap Image Failure (25)


In [30]:
c.laser = "open"

In [20]:
c.right_filter = "620"

In [32]:
c.snap()

array([[123, 108, 118, ..., 127, 139, 133],
       [129, 131, 130, ..., 127, 139, 122],
       [135, 120, 140, ..., 124, 131, 136],
       ...,
       [120, 145, 104, ..., 141, 131, 125],
       [124, 141, 112, ..., 126, 125, 147],
       [132, 132, 128, ..., 133, 110, 127]], dtype=uint16)

In [109]:
# Want to have an easy way to specify exposures across channels
c.exposure = 100

Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)


In [6]:
viewer, worker = gui.live(c)

In [30]:
c.mux

[1 0 0 0]
[0 0 0 0]
1
0
[1 0 0 0]
[1 0 0 0]
[0 0 0 0]
1
0
[1 0 0 0]


'invalid'

In [121]:
c.right_filter = "open"
c.top_filter = "open1"
c.laser = "closed"
c.light = 0
c.exposure = 10

## Tiling

In [48]:
c.up_filter = "cy5"

In [66]:
c.light = 100

In [120]:
c.right_filter = "620"
c.top_filter = "open1"
c.laser = "open"
c.light = 0
c.exposure = 100

Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)


In [112]:
top_left = c.xy

In [113]:
bottom_right = c.xy

In [60]:
c.right_filter = "open"

In [87]:
top_channels = ["cy5"]
right_channels = ["620"]

In [96]:
overlap_pixels = 100
um_per_pixel = 1.6137
img = c.snap()
img_height_pixels = img.shape[0]
img_width_pixels = img.shape[1]
img_height_um = um_per_pixel * img.shape[0]
img_width_um = um_per_pixel * img.shape[1]
overlap_um = overlap_pixels * um_per_pixel
delta_x = img_width_um - overlap_um
delta_y = img_height_um - overlap_um
xs = np.arange(top_left[0], bottom_right[0], delta_x)
ys = np.arange(top_left[1], bottom_right[1], delta_y)

canvas = np.zeros((len(top_channels) + len(right_channels),
                (img_height_pixels - overlap_pixels) * len(ys) + overlap_pixels,
                (img_width_pixels - overlap_pixels) * len(xs) + overlap_pixels), dtype=np.uint16)
viewer, layers = napari.imshow(canvas, channel_axis=0, contrast_limits=[0, 65535], name=top_channels + right_channels)
def update_canvas(d):
    d, i, j, c = d[0], d[1], d[2], d[3]
    row = i * (img_height_pixels - overlap_pixels)
    col = j * (img_width_pixels - overlap_pixels)
    viewer.layers[c].data[row:row + img_height_pixels, col:col + img_width_pixels] = d
    viewer.layers[c].refresh()

@thread_worker(connect={"yielded": update_canvas})
def tile_images():
    c.uv_filter = "uv"
    for i, y in enumerate(ys):
        for j, x in enumerate(xs):
            c.xy = (x, y)
            c._core.waitForDevice("stage")
            c.right_filter = "open"
            c.laser = "closed"
            c.light = 100
            c.exposure = 1000
            time.sleep(3)
            for channel in top_channels:
                c.up_filter = channel
                time.sleep(1)
                yield (c.snap(), i, j, channel)
                c._core.waitForDevice("camera")
                time.sleep(1)

            c.up_filter = "open1"
            c.laser = "open"
            c.exposure = 1000
            c.light = 0
            for channel in right_channels:
                c.right_filter = channel
                time.sleep(1)
                yield (c.snap(), i, j, channel)
                c._core.waitForDevice("camera")
                time.sleep(1)
            time.sleep(1)

In [97]:
worker2 = tile_images()

In [19]:
worker2.quit()

In [18]:
c.devices[0].wait()

Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap

In [104]:
c.laser = "closed"

In [4]:
c.z = 5000

In [11]:
c.light = 0

In [3]:
c.exposure

3.0048

In [20]:
c.valves[47] = "open"

In [11]:
c.close()

In [33]:
for i in range(48):
    c.valves[i] = "closed"

for i in range(48):
    time.sleep(0.1)
    c.valves[i] = "open"

In [25]:
c.z = 5000

In [32]:
c.valves[9] = "closed"

In [102]:
layers[0].contrast_limits = [0, 65000]

In [103]:
layers[1].contrast_limits = [0, 65000]

In [115]:
c._core.isBufferOverflowed()

False

Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)


In [116]:
c._core.getBufferTotalCapacity()

0

Error in device "camera":  Snap Image Failure (25)


In [118]:
c._core.getBufferFreeCapacity()

0

Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)
Error in device "camera":  Snap Image Failure (25)


In [74]:
c.mux = "open"

[1 1 0 0]
[0 0 1 1]


In [73]:
c.mux = 3

[0 0 0 0]
[1 1 1 1]


In [63]:
c.mux

[0 1 1 1]
[1 0 0 0]
[0 1 1 1]
[1 0 0 0]


8